In [120]:
import numgl

from ipycanvas import RoughCanvas
canvas = RoughCanvas()

canvas.stroke_rect(100, 100, 100, 100)
canvas.fill_rect(50, 50, 100, 100)

canvas.stroke_circle(300, 300, 100)
canvas.fill_circle(350, 350, 100)

canvas.stroke_line(200, 200, 300, 300)

canvas


RoughCanvas()

In [154]:
import numpy as np
from ipywidgets import interact
from ipycanvas import hold_canvas, RoughCanvas, Canvas

canvas = RoughCanvas()
width, height = 320, 200

n = 10
x, y = np.meshgrid(np.linspace(0, width, n), np.linspace(0, height, n))
w = np.ones(x.shape)
p1s = np.stack([x, y, w], axis=-1).reshape((-1, 3))

# external camera
def camera(f, ay, x, y):
    R = np.array([
        [np.cos(ay), 0, np.sin(ay)],
        [0, 1, 0],
        [-np.sin(ay), 0, np.cos(ay)],
    ])
    t = np.array([x*width, y*height, -500])
    #t = np.array([0,0,-500])
    K = np.array([
        [f, 0, 0],
        [0, f, 0],
        [0, 0, 1],
    ])
    #t = -np.dot(R, t)
    return np.dot(K, np.hstack([R, t[:, None]]))
    #return np.hstack([np.dot(K, R), t[:, None]])


def homogenize(euclidian):
    return np.vstack((euclidian, np.ones((1, euclidian.shape[1]))))


def dehomogenize(homogenous):
    return homogenous[:-1, :] / homogenous[-1, :]


def stroke_line(canvas, p1, p2):
    scale = np.array([50, 50, 1])
    translate = np.array([canvas.width/2, canvas.height/2, 0])
    #translate = np.array([0, 0])
    p1 = scale * p1 + translate
    p2 = scale * p2 + translate
    canvas.stroke_line(p1[0], p1[1], p2[0], p2[1])

    
def stroke_lines(canvas, mvp, lines):
    # split into from and to coordinates per line
    p1s, p2s = np.split(lines, 2, axis=1)
    p1s = np.squeeze(p1s, axis=1)
    p2s = np.squeeze(p2s, axis=1)
    # project with camera
    pp1s = dehomogenize(np.dot(mvp, homogenize(p1s.T))).T
    pp2s = dehomogenize(np.dot(mvp, homogenize(p2s.T))).T

    with hold_canvas(canvas):
        for p1, p2 in zip(pp1s, pp2s):
            stroke_line(canvas, p1, p2)


def rectangle(width, height):
    """Create lines for a rectangle"""
    return np.array([
        [[0, 0, 0], [width, 0, 0]],
        [[width, 0, 0], [width, height, 0]],
        [[width, height, 0], [0, height, 0]],
        [[0, height, 0], [0, 0, 0]],
    ])


def update_focal(f, ay):
    K = np.array([
        [f, 0, width/2],
        [0, f, height/2],
        [0, 0, 1],
    ])
    normals = np.dot(np.linalg.inv(K), p1s.T).T
    p2s = p1s + normals * 50
    lines = np.stack([p1s, p2s], axis=1)  # same shape as matplotlib

    # pose
    target = np.array([width/2, height/2, 0])
    up = np.array([0, 1, 0])
    r = 600
    t = ay
    eye = np.array([r * np.sin(t), 0, r * np.cos(t)])
    pose = numgl.lookat(eye, target, up)

    mvp = np.dot(numgl.perspective(90, canvas.width/canvas.height, 0.1, 5), pose)
    canvas.clear() 
    stroke_lines(canvas, mvp, lines)
    stroke_lines(canvas, mvp, rectangle(width, height))
    
interact(update_focal, f=(50, 1000), ay=(0, 2*np.pi))
canvas


interactive(children=(IntSlider(value=525, description='f', max=1000, min=50), FloatSlider(value=3.14159265358…

RoughCanvas()